In [ ]:
!pip install pyperclip
!pip install tabulate
!pip install requests
!pip install selenium
!pip install BeautifulSoup

In [3]:
import time
import requests
import urllib.request
from tabulate import tabulate

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyperclip

def clipboard_input(user_xpath, user_input):
        temp_user_input = pyperclip.paste()  # 사용자 클립보드를 따로 저장

        pyperclip.copy(user_input)
        driver.find_element(By.XPATH, user_xpath).click()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

        pyperclip.copy(temp_user_input)  # 사용자 클립보드에 저장 된 내용을 다시 가져 옴
        time.sleep(0.5)

In [4]:
###############
## 기본정보 세팅후 네이버 로그인하기

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
driver.implicitly_wait(5)
driver.get('https://nid.naver.com/nidlogin.login')
login = {
    "id" : "아이티",
    "pw" : "비밀번호"
}

# 아이디와 비밀번호를 입력합니다.
time.sleep(0.5) ## 0.5초
clipboard_input('//*[@id="id"]', login.get("id"))
clipboard_input('//*[@id="pw"]', login.get("pw"))
driver.find_element(By.XPATH, '//*[@id="log.login"]').click()

C:\Users\User\AppData\Local\Temp\ipykernel_1660\893207449.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
C:\Users\User\AppData\Local\Temp\ipykernel_1660\893207449.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


In [5]:
###############
## 기본정보 세팅후 강남구청 로그인하기

아동이름 = '서진영'
아동생년월일 = '2019-01-17'
휴대폰번호 = '010-6685-0501'
인원수 = '1'
거주지역 = '장지동' 

#/resv/apply/segok_indoor_playground/list.do?sch_week_day=2022-10&mid=ID04_04070903

## 도곡놀이터
#PLAYGROUND = ['https://www.gangnam.go.kr/resv/apply/memewe_clean_playground/list.do?mid=ID04_02071902'
#             ,]

## 세곡놀이터
PLAYGROUND = ['https://www.gangnam.go.kr/resv/apply/segok_indoor_playground/list.do?mid=ID04_04070903'
              ,'https://www.gangnam.go.kr/resv/reqst/segok_indoor_playground//insert.do?']

## 일원놀이터
#PLAYGROUND = ['https://www.gangnam.go.kr/resv/apply/irwon_playground/list.do?mid=ID04_04074703']  

PLAY = 'playground/'
INSERT = '/insert.do?'
LOGIN = 'https://www.gangnam.go.kr/login.do?mid=ID91_9101&returnUrl='+PLAYGROUND[0][PLAYGROUND[0].index('go.kr')+5:]
driver.get(LOGIN)
driver.find_element(By.XPATH, '//*[@id="contents-wrap"]/div[1]/div/div/div/div[1]/div[2]/div/div/ul/li[1]/button').click()

In [6]:
###############
## 로그인한 정보로 예약 목록 불러오기 : 11시 정각에 실행해야 할것

headers = {
"User-Agent":
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
}
ss = requests.session()
ss.headers.update(headers)

for cookie in driver.get_cookies():
    c = {cookie['name']: cookie['value']}
    ss.cookies.update(c)

res = ss.get(PLAYGROUND[0])
reserve = bs(res.text, "html.parser")
table = [['날짜','예약시간','예약번호','예약시간','예약번호','예약시간','예약번호']]
for item in reserve.find_all("td"):
    add_item = []
    if item.find('div', 'fc-sat') != None or item.find('div', 'holiday-color') != None:
        #주말
        #print(item.select_one('span').get_text(), '일')    
        #for link in item.select('a'):
        #    print(link.get_text().strip(), link['href'][link['href'].find('playground/')+len('playground/'):link['href'].find('/add.do')])
        pass
    else:
        #평일
        add_item.append(item.select_one('span').get_text()+'일')
        for link in item.select('a'):
            add_item.append(link.get_text().strip())
            add_item.append(link['href'][link['href'].find('playground/')+len('playground/'):link['href'].find('/add.do')])
    if len(add_item) > 1 : table.append(add_item)

print(tabulate(table))

----  ---------------  --------  ---------------  --------  --------  --------
날짜  예약시간         예약번호  예약시간         예약번호  예약시간  예약번호
8일   13:30 (0 / 20)   1507      16:00 (16 / 20)  1508
9일   16:00 (9 / 20)   1509
10일  13:30 (5 / 20)   1510      16:00 (13 / 20)  1511
15일  16:00 (17 / 20)  1519
16일  16:00 (14 / 20)  1520
17일  16:00 (10 / 20)  1522
----  ---------------  --------  ---------------  --------  --------  --------


In [7]:
###################################################
#상위 목록에서 예약번호를 복사해서 값 변경후 실행  #
#######                                     #######
예약번호를넣어주세요 = '1522'               #######
#######                                     #######
###################################################

REQ_STR = []
REQ_STR.extend(PLAYGROUND[1][:PLAYGROUND[1].index(PLAY)+len(PLAY)] + 예약번호를넣어주세요 + PLAYGROUND[1][PLAYGROUND[1].index(INSERT):])
REQ_STR.extend(PLAYGROUND[0][PLAYGROUND[0].index('mid='):])
REQ_STR.extend('&rar_user_name='+아동이름+'&rar_hp_no='+휴대폰번호+'&rar_person_cnt='+인원수)
REQ_STR.extend('&rar_optn_keys=1&rar_optn_values='+아동생년월일+'&rar_optn_keys=2&rar_optn_values='+거주지역)
REQ_STR.extend('&rar_reqst_key=0&rar_file_key=&rar_sms_yn=Y&rar_agree_yn=Y&rar_terms_yn=Y')

result = ss.post(''.join(REQ_STR))
print(result)

<Response [200]>


In [9]:
confirm_table = [['순번,','예약일시','취소번호','신청자명','휴대폰번호','신청인원','신청인원','상태','등록일', '취소번호']]
res_list = ss.post(PLAYGROUND[0].replace('apply', 'reqst'))
confirm_list = bs(res_list.text, "html.parser")
for item in confirm_list.find_all('tr')[1:]:
    add_item = []
    for detail in item.find_all('td'):
        if len(detail.select('a'))>0 :
            cancel_str = str(detail.select('a')[0])
            cancel_str = cancel_str[cancel_str.index('playground/')+11:cancel_str.index('/view.do')]
            add_item.append(cancel_str)
        add_item.append(detail.text.strip())
    confirm_table.append(add_item)
print(tabulate(confirm_table))

-----  ----------------  ---------  --------  -------------  --------  --------  ----------  ------  --------
순번,  예약일시          취소번호   신청자명  휴대폰번호     신청인원  신청인원  상태        등록일  취소번호
1      2022-11-17 16:00  1522/9553  서진영    010-6685-0501  1         신청      2022-11-08
-----  ----------------  ---------  --------  -------------  --------  --------  ----------  ------  --------


In [10]:
###################################################
#상위 목록에서 취소번호를 복사해서 값 변경후 실행  #
#######                                     #######
취소번호를넣어주세요 = '1522/9553'          #######
#######                                     #######
###################################################

REQ_STR = []
REQ_STR.extend(PLAYGROUND[1][:PLAYGROUND[1].index(PLAY)+len(PLAY)] + 취소번호를넣어주세요 + '/cancel.do?')
REQ_STR.extend(PLAYGROUND[0][PLAYGROUND[0].index('mid='):])

result = ss.post(''.join(REQ_STR))
print(result)

<Response [200]>
